# 🤖 Entrenamiento de Modelo RoBERTa para Clasificación de Texto

Este notebook entrena un modelo RoBERTa fine-tuned para clasificar consultas de texto.

## 📋 Contenido
1. Instalación de dependencias
2. Carga y preprocesamiento de datos
3. Tokenización
4. Configuración y entrenamiento del modelo
5. Evaluación y métricas
6. Guardado del modelo
7. Inferencia

## 1️⃣ Instalación de Dependencias

Instalamos las librerías necesarias con versiones compatibles.

In [ ]:
# Desinstalar versiones conflictivas
!pip uninstall -y pyarrow apache-beam

# Instalar versiones específicas compatibles
!pip install -U "pyarrow==16.1.0" "pandas==2.2.2" "datasets==2.19.1"
!pip install -U "transformers>=4.41.0" "sentence-transformers>=2.5.1" datasets

## 2️⃣ Importación de Librerías

In [ ]:
# Librerías estándar
import os
import json
import unicodedata
import re

# Análisis de datos
import pandas as pd
import numpy as np

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Transformers
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# Desactivar Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

print("✅ Librerías importadas correctamente")

## 3️⃣ Configuración de Rutas

**IMPORTANTE:** Modifica estas rutas según tu sistema local.

In [ ]:
# 📁 CONFIGURACIÓN DE RUTAS - MODIFICAR SEGÚN TU SISTEMA
DATA_PATH = './data/consultas_modelo_ia.xlsx'  # Ruta al archivo Excel con los datos
MODEL_SAVE_PATH = './modelo_entrenado'  # Carpeta donde se guardará el modelo
PREDICTIONS_PATH = './predicciones.xlsx'  # Archivo de salida con predicciones

# Crear carpeta para el modelo si no existe
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

print(f"📂 Ruta de datos: {DATA_PATH}")
print(f"💾 Modelo se guardará en: {MODEL_SAVE_PATH}")
print(f"📊 Predicciones se guardarán en: {PREDICTIONS_PATH}")

## 4️⃣ Carga y Preprocesamiento de Datos

Cargamos el archivo Excel local y limpiamos el texto.

In [ ]:
# Función de limpieza de texto
def limpiar_texto(texto):
    """Limpia y normaliza el texto de entrada."""
    texto = str(texto).lower()
    # Normalizar caracteres Unicode
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Eliminar caracteres especiales excepto puntuación básica
    texto = re.sub(r"[^a-z0-9¿?¡!., ]", " ", texto)
    # Eliminar espacios múltiples
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

# Cargar datos desde archivo Excel local
print("📥 Cargando datos...")
df = pd.read_excel(DATA_PATH)

print(f"✅ Datos cargados: {len(df)} registros")
print(f"📋 Columnas disponibles: {df.columns.tolist()}")

# Aplicar limpieza
df['cns_descripcion'] = df['cns_descripcion'].fillna('').apply(limpiar_texto)
df['clasificaciones'] = df['clasificaciones'].astype(str)

# Codificar etiquetas como números
df['clasificacion_encoded'] = df['clasificaciones'].astype('category').cat.codes

# Obtener categorías
categorias = df['clasificaciones'].astype('category').cat.categories.to_list()
print(f"\n🏷️  Categorías encontradas: {categorias}")
print(f"📊 Distribución de clases:\n{df['clasificaciones'].value_counts()}")

# Mostrar ejemplos
print("\n📝 Ejemplos de datos:")
print(df[['cns_descripcion', 'clasificaciones']].head())

## 5️⃣ División del Dataset

Dividimos los datos en conjuntos de entrenamiento (80%), validación (10%) y prueba (10%).

In [ ]:
# Extraer textos y etiquetas
data_texts = df['cns_descripcion'].to_list()
data_labels = df['clasificacion_encoded'].to_list()

# División estratificada
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data_texts, data_labels, 
    test_size=0.2, 
    stratify=data_labels, 
    random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, 
    test_size=0.5, 
    random_state=42
)

print("📊 Distribución de datos:")
print(f"  🔹 Entrenamiento: {len(train_texts)} textos ({len(train_texts)/len(data_texts)*100:.1f}%)")
print(f"  🔹 Validación:    {len(val_texts)} textos ({len(val_texts)/len(data_texts)*100:.1f}%)")
print(f"  🔹 Prueba:        {len(test_texts)} textos ({len(test_texts)/len(data_texts)*100:.1f}%)")

## 6️⃣ Tokenización

Utilizamos el tokenizador de RoBERTalex (modelo en español).

In [ ]:
# Cargar tokenizador
print("🔤 Cargando tokenizador RoBERTalex...")
tokenizer = RobertaTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')

# Tokenizar conjuntos de datos
MAX_LENGTH = 256

print(f"⚙️  Tokenizando con longitud máxima: {MAX_LENGTH}")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LENGTH)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LENGTH)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=MAX_LENGTH)

# Crear clase Dataset personalizada
class EmailDataset(torch.utils.data.Dataset):
    """Dataset personalizado para clasificación de texto."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets
train_dataset = EmailDataset(train_encodings, train_labels)
val_dataset = EmailDataset(val_encodings, val_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

print("✅ Tokenización completada")

## 7️⃣ Configuración del Modelo

Calculamos pesos de clase para balancear el entrenamiento y configuramos el modelo.

In [ ]:
# Calcular pesos de clase para balancear
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"🖥️  Dispositivo: {device}")
print(f"⚖️  Pesos de clase: {class_weights}")

# Cargar modelo preentrenado
print("\n🤖 Cargando modelo RoBERTalex...")
model = RobertaForSequenceClassification.from_pretrained(
    'PlanTL-GOB-ES/RoBERTalex',
    num_labels=len(categorias)
).to(device)

print("✅ Modelo cargado correctamente")

## 8️⃣ Trainer Personalizado con Pesos de Clase

In [ ]:
# Función de métricas
def compute_metrics(pred):
    """Calcula F1-score ponderado."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return {"f1": f1}

# Trainer con pesos de clase
class WeightedTrainer(Trainer):
    """Trainer personalizado que aplica pesos de clase en la función de pérdida."""
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

print("✅ Trainer personalizado configurado")

## 9️⃣ Configuración de Entrenamiento

In [ ]:
# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none",
    save_strategy="epoch",
    evaluation_strategy="epoch"
)

# Instanciar trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("✅ Configuración de entrenamiento lista")

## 🔟 Entrenamiento del Modelo

**NOTA:** Este proceso puede tardar varios minutos dependiendo del hardware.

In [ ]:
print("🚀 Iniciando entrenamiento...\n")
trainer.train()
print("\n✅ Entrenamiento completado")

## 1️⃣1️⃣ Evaluación en Conjunto de Prueba

In [ ]:
# Realizar predicciones
print("🔍 Evaluando modelo en conjunto de prueba...")
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()
y_true = test_labels

# Reporte de clasificación
print("\n📊 Reporte de Clasificación:\n")
print(classification_report(y_true, y_pred, target_names=categorias))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=categorias, yticklabels=categorias)
plt.xlabel('Clase Predicha')
plt.ylabel('Clase Real')
plt.title('Matriz de Confusión')
plt.tight_layout()
plt.show()

print("\n✅ Evaluación completada")

## 1️⃣2️⃣ Guardar Modelo y Resultados

In [ ]:
# Guardar modelo y tokenizador
print(f"💾 Guardando modelo en: {MODEL_SAVE_PATH}")
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

# Guardar etiquetas
with open(f"{MODEL_SAVE_PATH}/labels.json", "w", encoding='utf-8') as f:
    json.dump(categorias, f, ensure_ascii=False, indent=2)

print("✅ Modelo guardado correctamente")

# Exportar predicciones
df_pred = pd.DataFrame({
    "Texto": test_texts,
    "Clase Real": [categorias[i] for i in y_true],
    "Clase Predicha": [categorias[i] for i in y_pred]
})

df_pred.to_excel(PREDICTIONS_PATH, index=False)
print(f"📊 Predicciones exportadas a: {PREDICTIONS_PATH}")

## 1️⃣3️⃣ Inferencia - Ejemplo de Uso

Prueba el modelo entrenado con un texto de ejemplo.

In [ ]:
def predecir_texto(texto, modelo, tokenizador, categorias, max_length=256):
    """Predice la categoría de un texto."""
    # Limpiar texto
    texto_limpio = limpiar_texto(texto)
    
    # Tokenizar
    inputs = tokenizador(
        texto_limpio, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=max_length
    ).to(device)
    
    # Predecir
    modelo.eval()
    with torch.no_grad():
        outputs = modelo(**inputs)
    
    # Obtener clase predicha y confianza
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, axis=1).item()
    confidence = probs[0][predicted_class].item()
    
    return categorias[predicted_class], confidence

# Ejemplo de uso
texto_ejemplo = "Buenos días, quisiera solicitar una reunión para revisar mis asignaturas pendientes"

categoria, confianza = predecir_texto(texto_ejemplo, model, tokenizer, categorias)

print("\n🔮 Predicción:")
print(f"  📝 Texto: {texto_ejemplo}")
print(f"  🏷️  Categoría: {categoria}")
print(f"  📊 Confianza: {confianza:.2%}")

## 1️⃣4️⃣ Clasificación por Lotes (Opcional)

Clasifica múltiples textos de un archivo Excel.

In [ ]:
# OPCIONAL: Clasificar archivo Excel completo
# Descomenta y modifica la ruta si necesitas clasificar un archivo nuevo

# input_excel = './data/consultas_nuevas.xlsx'
# output_excel = './consultas_clasificadas.xlsx'

# df_nuevas = pd.read_excel(input_excel)

# def predecir_batch(texto):
#     categoria, _ = predecir_texto(texto, model, tokenizer, categorias)
#     return categoria

# df_nuevas['clasificacion_predicha'] = df_nuevas['cns_descripcion'].apply(predecir_batch)
# df_nuevas.to_excel(output_excel, index=False)

# print(f"✅ Clasificación por lotes completada: {output_excel}")

---

## 🎉 ¡Entrenamiento Completado!

### 📦 Archivos Generados:
- **Modelo entrenado:** `{MODEL_SAVE_PATH}/`
- **Predicciones:** `{PREDICTIONS_PATH}`

### 🚀 Próximos Pasos:
1. Integrar el modelo en la API Flask
2. Realizar pruebas con datos reales
3. Ajustar hiperparámetros si es necesario
4. Desplegar en producción